In [1]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import seaborn as sns
sns.set()
import gc
gc.collect()
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
from IPython.display import clear_output
#set_matplotlib_formats('retina')
#from tqdm import tqdm
#tqdm.pandas()
import numpy as np
from pyhive import presto
from datetime import datetime, timedelta
#from bson import ObjectId
from functools import reduce
from sklearn.cluster import KMeans
#from pymongo import MongoClient
import glob
import warnings
warnings.filterwarnings('ignore')
from datetime import date
import json
#import re
#import dtale
#from h3 import h3
# import pandasql as ps

In [3]:
metabase_connection = presto.connect(
        #host='bi-trino-2.serving.data.production.internal',
        #host='bi-trino.serving.data.production.internal',
        #host='bi-presto.serving.data.production.internal',
        #host='prime-trino.serving.data.production.internal',
        #"presto.processing.yoda.run",
        host='bi-trino-2.serving.data.production.internal',
        port=80,
        protocol='http',
        catalog ='hive',
        username='praveen.u@rapido.bike')

In [4]:
#Monitor new (not active in the last 1 month from launch) and existing Online Caps' - O2n and APR

In [25]:
grs_odd_even=pd.DataFrame()

for i in pd.date_range('20230501','20230511'):

    query="""with main_table as (select city,captain_id,sum(net_rides_taxi) as total_net_rides
                    from datasets.captain_metrics 
                    where total_login_hours>0 
                    and yyyymmdd>= '20230404' and yyyymmdd <= '20230504' 
                    and service_mode='auto' and city in ('Chennai','Hyderabad','Bangalore')
                    group by 1,2

                    ),


    c_type as (select city,captain_id,

                        case when total_net_rides=0 then 'Online' 
                             when total_net_rides!=0 then 'Online_net' 
                            else 'NA'
                            end as cap_type

                        from main_table
                    ),


    agg_online as ( select city,cap_type,count( distinct captain_id) as total_one_month_caps 

                     from c_type
                     group by 1 ,2
                    ),

    order_filtering as 
        (
            select distinct order_id

            from 
                orders.order_logs_snapshot
                where yyyymmdd = cast({dt} as varchar)
                and service_obj_service_name = 'Auto'
                and (spd_fraud_flag = false OR spd_fraud_flag IS NULL)
                and lower(trim(city_name)) in ('hyderabad','chennai','bangalore')
        ),

    rider_pings as
        (

                select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,
                concat(data_propagationBatchId,data_riderId,data_orderid) as unique_ping_level_id,
                concat(data_orderid,data_riderId) as unique_id_order_level_id,epoch
                from raw.kafka_dispatch_propagation_immutable
                where yyyymmdd = cast({dt} as varchar)
                --where yyyymmdd = '20230506'

                    and data_orderid in (select distinct order_id from order_filtering)
                    and data_propagationtype in ('multicast')
                    and eventtype in ('rider_acknowledged')


                union all

                select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,
                concat(data_propagationBatchId,data_riderId,data_orderid) as unique_ping_level_id,
                concat(data_orderid,data_riderId) as unique_id_order_level_id,epoch
                from(
                select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,epoch,
                row_number() over(partition by data_propagationBatchId,data_riderId,data_orderid order by epoch desc) as row,data

                from raw.kafka_dispatch_propagation_immutable
                where yyyymmdd = cast({dt} as varchar)
                --where yyyymmdd = '20230506'

                    and data_orderid in (select distinct order_id from order_filtering)
                    and data_propagationtype in ('multicast')
                    and eventtype in ('rider_busy','order_already_accepted','rider_accept_failed','rider_rejected','rider_accepted','rider_ineligible','rider_unreachable')
                    ) base
                where row =1

        )

                select  city,yyyymmdd,cap_type,total_one_month_caps,Online_Cap,Net_Cap,
                (Net_Cap/cast(Online_Cap as real))*100 as O2N_percentage,
                (acc_pings/cast(total_pings as real))*100 as APR

                from
               (
                select t1.city,t1.yyyymmdd,t3.cap_type,total_one_month_caps, sum(Online_Cap) as Online_Cap, sum(Net_Cap) as Net_Cap,sum(accepted_pings+already_accepted_intent_pings) as acc_pings,sum(total_pings) as total_pings
                from(
                 select distinct city,yyyymmdd,captain_id,
                 case when total_login_hours>0 then 1 else 0 end as Online_Cap,
                 case when net_rides_taxi>0 then 1 else 0 end as Net_Cap

                from datasets.captain_metrics 

                where total_login_hours>0 and yyyymmdd>= cast({dt} as varchar) and yyyymmdd<= cast({dt} as varchar) and service_mode='auto' 
                and city in ('Chennai','Hyderabad','Bangalore')
                    ) as t1
                    left join c_type t2 
                    on t1.captain_id = t2.captain_id
                    and t1.city = t2.city

                    left join agg_online t3
                    on t3.city=t2.city and t3.cap_type= t2.cap_type


                    left join 
                    (

                        select yyyymmdd,data_riderId,
                          count(distinct case when eventtype='rider_accepted' then unique_ping_level_id end) as accepted_pings,
                            count(distinct case when eventtype='rider_accept_failed' then unique_ping_level_id end) as already_accepted_intent_pings,
                            count(distinct case when eventtype='rider_acknowledged' then unique_ping_level_id end) as total_pings

                            from rider_pings

                            group by 1,2 

                    ) as rp 

                    on rp.yyyymmdd= t1.yyyymmdd and rp.data_riderId = t1.captain_id


                    where t3.cap_type!=''
                    group by 1,2,3,4 


               )  


                where city='Bangalore'
                order by 1,2,3 

                """.format(dt=i.strftime('%Y%m%d'))

    df_test = pd.read_sql_query(query, metabase_connection)

    print(i)
    
    grs_odd_even=grs_odd_even.append(df_test)




2023-05-01 00:00:00


DatabaseError: {'message': 'Query exceeded distributed user memory limit of 75GB', 'errorCode': 131073, 'errorName': 'EXCEEDED_GLOBAL_MEMORY_LIMIT', 'errorType': 'INSUFFICIENT_RESOURCES', 'failureInfo': {'type': 'io.trino.ExceededMemoryLimitException', 'message': 'Query exceeded distributed user memory limit of 75GB', 'suppressed': [], 'stack': ['io.trino.ExceededMemoryLimitException.exceededGlobalUserLimit(ExceededMemoryLimitException.java:29)', 'io.trino.memory.ClusterMemoryManager.process(ClusterMemoryManager.java:225)', 'io.trino.execution.SqlQueryManager.enforceMemoryLimits(SqlQueryManager.java:297)', 'io.trino.execution.SqlQueryManager.lambda$start$0(SqlQueryManager.java:104)', 'java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:515)', 'java.base/java.util.concurrent.FutureTask.runAndReset(FutureTask.java:305)', 'java.base/java.util.concurrent.ScheduledThreadPoolExecutor$ScheduledFutureTask.run(ScheduledThreadPoolExecutor.java:305)', 'java.base/java.util

In [26]:
df_test

,city,yyyymmdd,cap_type,total_one_month_caps,Online_Cap,Net_Cap,O2N_percentage,APR
0,Bangalore,20230501,Online,13546,1796,0,0.000000,1.253963
1,Bangalore,20230501,Online_net,44345,23598,12302,52.131535,2.947619


In [14]:
query="""with main_table as (select city,captain_id,sum(net_rides_taxi) as total_net_rides
                from datasets.captain_metrics 
                where total_login_hours>0 
                and yyyymmdd>= '20230404' and yyyymmdd <= '20230504' 
                and service_mode='auto' and city in ('Chennai','Hyderabad','Bangalore')
                group by 1,2
                 
                ),


c_type as (select city,captain_id,
                
                    case when total_net_rides=0 then 'Online' 
                         when total_net_rides!=0 then 'Online_net' 
                        else 'NA'
                        end as cap_type
                        
                    from main_table
                ),
            
                
agg_online as ( select city,cap_type,count( distinct captain_id) as total_one_month_caps 
            
                 from c_type
                 group by 1 ,2
                ),
                
order_filtering as 
    (
        select distinct order_id

        from 
            orders.order_logs_snapshot
            where yyyymmdd = '20230506'--cast({dt} as varchar)
            and service_obj_service_name = 'Auto'
            and (spd_fraud_flag = false OR spd_fraud_flag IS NULL)
            and lower(trim(city_name)) in ('hyderabad','chennai','bangalore')
    ),

rider_pings as
    (

            select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,
            concat(data_propagationBatchId,data_riderId,data_orderid) as unique_ping_level_id,
            concat(data_orderid,data_riderId) as unique_id_order_level_id,epoch
            from raw.kafka_dispatch_propagation_immutable
            --where yyyymmdd = cast({dt} as varchar)
            where yyyymmdd = '20230506'

                and data_orderid in (select distinct order_id from order_filtering)
                and data_propagationtype in ('multicast')
                and eventtype in ('rider_acknowledged')


            union all

            select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,
            concat(data_propagationBatchId,data_riderId,data_orderid) as unique_ping_level_id,
            concat(data_orderid,data_riderId) as unique_id_order_level_id,epoch
            from(
            select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,epoch,
            row_number() over(partition by data_propagationBatchId,data_riderId,data_orderid order by epoch desc) as row,data

            from raw.kafka_dispatch_propagation_immutable
            --where yyyymmdd = cast({dt} as varchar)
            where yyyymmdd = '20230506'

                and data_orderid in (select distinct order_id from order_filtering)
                and data_propagationtype in ('multicast')
                and eventtype in ('rider_busy','order_already_accepted','rider_accept_failed','rider_rejected','rider_accepted','rider_ineligible','rider_unreachable')
                ) base
            where row =1

    )
        
--            select  city,yyyymmdd,cap_type,total_one_month_caps,Online_Cap,Net_Cap,
 --           (Net_Cap/cast(Online_Cap as real))*100 as O2N_percentage
            
  --          from
   --         (
            select t1.city,t1.yyyymmdd,t3.cap_type,total_one_month_caps, sum(Online_Cap) as Online_Cap, sum(Net_Cap) as Net_Cap,sum(accepted_pings+already_accepted_intent_pings) as acc_pings,sum(total_pings) as total_pings
            from(
            
             select yyyymmdd,data_riderId,
                      count(distinct case when eventtype='rider_accepted' then unique_ping_level_id end) as accepted_pings,
                        count(distinct case when eventtype='order_already_accepted' then unique_ping_level_id end) as already_accepted_pings,
                        count(distinct case when eventtype='rider_accept_failed' then unique_ping_level_id end) as already_accepted_intent_pings,
                        count(distinct case when eventtype='rider_acknowledged' then unique_ping_level_id end) as total_pings
                        
                        from rider_pings
                        
                        group by 1,2 
                        
                ) as rp
                
                left join (
                    select distinct city,yyyymmdd,captain_id,
                     case when total_login_hours>0 then 1 else 0 end as Online_Cap,
                     case when net_rides_taxi>0 then 1 else 0 end as Net_Cap

                    from datasets.captain_metrics 

                    where total_login_hours>0 and yyyymmdd>='20230506' and yyyymmdd<='20230506' and service_mode='auto' 
                    and city in ('Chennai','Hyderabad','Bangalore')
                ) as t1
                
                on rp.yyyymmdd= t1.yyyymmdd and rp.data_riderId = t1.captain_id
                
                left join c_type t2 
                on t1.captain_id = t2.captain_id
                and t1.city = t2.city
                
                left join agg_online t3
                on t3.city=t2.city and t3.cap_type= t2.cap_type

                where t3.cap_type!=''
                group by 1,2,3,4 
                
                
          --  )  
            
            
         --   where city='Bangalore'
         --   order by 1,2,3 
            
            """
    
df_test = pd.read_sql_query(query, metabase_connection)

    

In [15]:
df_test

,city,yyyymmdd,cap_type,total_one_month_caps,Online_Cap,Net_Cap,acc_pings,total_pings
0,Hyderabad,20230506,Online_net,42056,21853,13974,182632,3125526
1,Hyderabad,20230506,Online,10776,1674,451,5116,119972
2,Chennai,20230506,Online,8617,1579,350,2439,56102
3,Bangalore,20230506,Online_net,44345,22453,13063,125490,3975013
4,Chennai,20230506,Online_net,30349,17297,9544,68965,1226886
5,Bangalore,20230506,Online,13546,1766,409,3300,131015
